In [1]:
import pandas as pd
import numpy  as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lindachen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

#file = open('../Resources/' + filename, 'r', encoding="ISO-8859-1")
df = pd.read_csv("data/EA_CSV.csv", engine="python", encoding="UTF-8")

In [ ]:
#turn the first row into column
def get_data(df):
    #combine the description
    df['description'] = df['Expertise'].fillna('') + " " + df['Title'].fillna('')
    return df

In [7]:
def clean_text(s):
    #removeNonAscii
    s = "".join(i for i in s if ord(i)<128)

    #return all lower cases
    s = s.lower()

    #remove stop wrods
    s = s.split()
    stops = set(stopwords.words("english"))
    text = [w for w in s if not w in stops]
    text = " ".join(text)

    #remove html
    html_pattern = re.compile('<.*?>')
    text = html_pattern.sub(r'', text)

    #remove punctuation
    text = re.sub(r'[^\w\s]'," ",text)
    return text



In [ ]:
def glove_model(file_path='glove.twitter.27B.200d.txt'):
  """
  input: the access to the file which contains the pre-trained glove model
  ouput: a trained model using glove
  """

  embeddings_index = {}
  f = open(file_path, encoding='utf-8')
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()
  return embeddings_index

In [ ]:
def glove2vec(text, model):
    """
    input: 
    - text: the string that you wanted to turn into vectors
    - model: the glove pre-trained model that you wanted to use

    output:
    an average vector of your input text using the glove model

    """
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    vectors = {}

    avgword2vec = None
    count = 0

    for word in text.split():
        if word in model:
            count += 1
            if avgword2vec is None:
                avgword2vec = model[word]
            else:
                avgword2vec = avgword2vec + model[word]
            
    if avgword2vec is not None:
        avgword2vec = avgword2vec / count
    
    word_embeddings.append(avgword2vec)

    return word_embeddings

In [ ]:
    #df = get_data(df)
    #model = pd.read_pickle("models/glove_model.pkl")
    #df['description'] = df['description'].apply(clean_text)
    #EAs_vectors = [glove2vec(EA, model) for EA in df['description']]
    pd.to_pickle(EAs_vectors, "./EAvector.pkl")

In [ ]:
def recommendations_glove(input, df=df, col='description'):
    """
    input: 
    - input: key words relating to the article
    - df: the dataframe that we are using
    - col: the column name that contains description of EAs
    - file path: the file path to the pre-trained glove model

    output:
    a list of EAs' names ranked by the most recommended / most suitable EA
    to the least
    """

    #get the model (it is too slow to compute the model every time)
    model = glove_model(file_path='glove.twitter.27B.200d.txt')

    # vecterizd the input
    vector_input = glove2vec(input, model)

    # vecterized all the descriptions of EAs
    df[col] = df[col].apply(clean_text)
    EAs_vectors = [glove2vec(EA, model) for EA in df[col]]

    #drop an EA if that EA has "None" vector
    df['vectors'] = EAs_vectors
    for i, n in enumerate(df['vectors']):
      if n[0] is None:
        df = df.drop([i+1], axis=0)


    # finding cosine similarity for the vectors
    similarity = []
    for n in df['vectors']:
      scores = cosine_similarity(vector_input, n)[0][0]
      similarity.append(scores)
    
    df['similarity'] = similarity


    # sort and find the recommended movie
    df = df.sort_values(by=['similarity'], ascending=False)
    #res_df = df.iloc[:m]


    return df['Names']

In [2]:
import joblib

In [3]:
    vec_df = pd.read_pickle('models/vectors.pkl')

    model = joblib.load('models/glove_model.pkl')
    keywords = 'data collection, data ethics issues/initial assumptions'

In [4]:
def input_vec(input, model):
    avgword2vec = None
    count = 0
    for word in input.split():
        if word in model:
            count += 1
            if avgword2vec is None:
                avgword2vec = model[word]
            else:
                avgword2vec = avgword2vec + model[word]

    if avgword2vec is not None:
        avgword2vec = avgword2vec / count

    return avgword2vec

In [19]:
input_vec(keywords, model)

array([-0.04264499, -0.2412725 , -0.19788952,  0.2706835 ,  0.09025126,
        0.13406649,  0.5248575 , -0.04231749, -0.18978775, -0.27109   ,
        0.256518  ,  0.01726   , -0.4055375 , -0.37101498,  0.08154001,
       -0.224914  , -0.4646575 , -0.0740575 , -0.11195499, -0.55616   ,
       -0.23919249, -0.02768325,  0.101285  ,  0.1464285 , -0.2176025 ,
        0.75134754,  0.31227002, -0.37628752, -0.21252   ,  0.30583948,
       -0.0733735 , -0.26635998, -0.11816249, -0.1986675 ,  0.37949753,
       -0.0750975 , -0.11551675,  0.14888625, -0.43187824,  0.12724748,
        0.066405  ,  0.32920498,  0.16730751,  0.26131675,  0.3879875 ,
        0.02510749, -0.02189072,  0.1108    ,  0.12375499, -0.2491525 ,
        0.76581   ,  0.57555246,  0.01829675,  0.23388399, -0.1911175 ,
       -0.18047   , -0.5476675 ,  0.22015168, -0.3309175 , -0.3405975 ,
        0.1253877 ,  0.39254498,  0.3417295 ,  0.47351003,  0.47570974,
        1.0514901 ,  0.361315  , -0.6885093 , -0.0572475 ,  0.25

In [22]:
def recomm_engine(keywords, vec_df, model):
    # vectorize the input
    cleaned_input = clean_text(keywords)
    input_vector = input_vec(cleaned_input, model)

    # finding cosine similarity for the vectors
    similarity = []
    input_vector = input_vector.reshape(1,-1)
    for n in vec_df['vectors']:
        n = n.reshape(1,-1)
        scores = cosine_similarity(input_vector, n)[0][0]
        similarity.append(scores)

    vec_df['similarity'] = similarity

    # sort and find the recommended movie
    vec_df = vec_df.sort_values(by=['similarity'], ascending=False)

    return vec_df.loc[::, :'Title']

In [21]:
recomm_engine(keywords, vec_df, model)

,Names,\nAvailable,M,Tu,W,Th,F,Sa,Su,"TDS topics: DS, ML, Prog, Visu, AI (...)",Expertise,Connect,Title,Unnamed: 13,Our FAQs,description
23,Sergio Betancourt,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"Statistics, deep/generative/reinforcement lear...",https://www.linkedin.com/in/sergio-e-betancourt/,Data Science at Nulogy | Data Education at Bra...,NaN,NaN,statistics deep generative reinforcement lear...
26,Carlos Ortega,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"Data Ethics, Civic Tech",https://www.linkedin.com/in/soycarlos/,None,NaN,NaN,data ethics civic tech
27,Julia Nikulski,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,None,https://www.linkedin.com/in/julia-nikulski/,Data Science & Sustainability | Ex-bloomberg,NaN,NaN,data science sustainability ex bloomberg
17,Mael Fabien,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"Speech processing, Criminology, Statistics",https://www.linkedin.com/in/mael-fabien/,Ph.D. Student @ Idiap/EPFL | Data Scientist,NaN,NaN,speech processing criminology statistics ph ...
8,Arman Didandeh,x,x,None,NaN,NaN,NaN,NaN,NaN,yes (will not be able to leave private notes f...,"NLP, Visual Analytics, Interaction design, Hum...",https://www.linkedin.com/in/armandidandeh/,Data Science + HCI (cloud developer),NaN,NaN,nlp visual analytics interaction design hum...
4,Gitansh Khirbat,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"NLP, Statistics, Optimizations",https://www.linkedin.com/in/gkhirbat/,Machine Learning Engineer | Applied Data Science,NaN,NaN,nlp statistics optimizations machine learnin...
5,Abdullah Farouk,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,deep learning,https://www.linkedin.com/in/abdullah-farouk/,Data Scientist at LoyaltyOne,NaN,NaN,deep learning data scientist loyaltyone
20,Rohan Joseph,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"optimization, scheduling",https://www.linkedin.com/in/rohan-joseph-b39a8...,Data Scientist at Apple,NaN,NaN,optimization scheduling data scientist apple
21,John Jagtiani,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"Management, Societal impacts",https://www.linkedin.com/in/jjagtiani/,"Group Vice President, CX Growth & Strategy at ...",NaN,NaN,management societal impacts group vice presid...
16,Lester Leong,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,Finance,https://www.linkedin.com/in/lester-leong-mba-m...,Data Science Leader | Consultant,NaN,NaN,finance data science leader consultant


In [23]:
vec_df

,Names,\nAvailable,M,Tu,W,Th,F,Sa,Su,"TDS topics: DS, ML, Prog, Visu, AI (...)",Expertise,Connect,Title,Unnamed: 13,Our FAQs,description,vectors,similarity
0,Amber Teng,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,None,https://www.linkedin.com/in/angelavteng/,DS | Quantitative Risk Modeling | Machine Lear...,NaN,NaN,ds quantitative risk modeling machine lear...,"[-0.17723532, -0.2903112, 0.068666, -0.0335756...",0.662083
1,Lowri Williams,x,x,None,NaN,NaN,NaN,NaN,NaN,yes,"Python, NLP",https://www.linkedin.com/in/lowriwilliams/,Research Data Scientist at Cardiff University,NaN,NaN,python nlp research data scientist cardiff un...,"[-0.06812172, -0.058114015, 0.03477508, -0.017...",0.705708
2,Sophie Mann,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"cognitive science/neuroscience, healthcare, ed...",https://www.linkedin.com/in/sophie-maya-mann/,Data Manager at Tennessee Department of Education,NaN,NaN,cognitive science neuroscience healthcare ed...,"[-0.02856793, -0.048768744, 0.0047871745, -0.0...",0.719291
3,Jingles,None,None,None,NaN,NaN,NaN,NaN,NaN,yes (no programming),in heath sector (eye),https://www.linkedin.com/in/jingles/,PhD Student | Alibaba Group | Healthcare (Eye ...,NaN,NaN,heath sector eye phd student alibaba group...,"[0.057771504, -0.043637745, -0.016351469, -0.0...",0.683416
4,Gitansh Khirbat,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"NLP, Statistics, Optimizations",https://www.linkedin.com/in/gkhirbat/,Machine Learning Engineer | Applied Data Science,NaN,NaN,nlp statistics optimizations machine learnin...,"[0.00071935257, -0.037235107, -0.003258852, -0...",0.757653
5,Abdullah Farouk,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,deep learning,https://www.linkedin.com/in/abdullah-farouk/,Data Scientist at LoyaltyOne,NaN,NaN,deep learning data scientist loyaltyone,"[-0.00890943, -0.007880044, 0.00040356503, 0.0...",0.751855
6,Andrew DeCotiis-Mauro,x,x,None,NaN,NaN,NaN,NaN,NaN,yes,None,https://www.linkedin.com/in/akdcm/,Software Developer | ETL | Analytics,NaN,NaN,software developer etl analytics,"[-0.0061842194, 0.0014298261, -0.015423066, -0...",0.521736
7,Anton Muehlemann,None,None,None,NaN,NaN,NaN,NaN,NaN,yes,"math, deep learning (computer vision and NLP)",https://www.linkedin.com/in/antonmu/,Mathematics PhD with specialization in non-con...,NaN,NaN,math deep learning computer vision nlp math...,"[-0.015092878, -0.022559721, 0.013245812, 0.00...",0.666206
8,Arman Didandeh,x,x,None,NaN,NaN,NaN,NaN,NaN,yes (will not be able to leave private notes f...,"NLP, Visual Analytics, Interaction design, Hum...",https://www.linkedin.com/in/armandidandeh/,Data Science + HCI (cloud developer),NaN,NaN,nlp visual analytics interaction design hum...,"[0.0019129494, -0.015345179, -0.023511717, -0....",0.759018
9,Carlos Mougan,x,None,x,NaN,NaN,NaN,NaN,NaN,yes,"explainability, xAI, fairness, ethics, shap, lime",https://www.linkedin.com/in/carlosmougan/,Statistician - Data Scientist Trainee at Europ...,NaN,NaN,explainability xai fairness ethics shap l...,"[-0.021173336, -0.013675677, -0.009887077, 0.0...",0.696109
